In [ ]:
pip install SimpleITK

In [ ]:
# IMPORTS #

import numpy as np
import matplotlib.pyplot as plt
import SimpleITK

In [ ]:
# MODULE IMPORTS # (because Jupyter NoteBooks can't import other Notebooks classes)

# Should be -----------------------
# import Image
#----------------------------------

from scipy.signal import wiener

def rgb2grey(rgb):
    """
        Creates a luma transformation of an rgb array
        
        :param rgb: a list of arrays
        :type rgb: numpy.ndarray
        :return: the grey array
        :rtype: numpy.ndarray
        
        :Example:
        
        >>> import numpy
        >>> a = numpy.array([1,2,3])
        >>> print(rgb2grey(a))
        [1.8596 1.8596 1.8596]
    """
    luma = rgb[0] * luma_coeff[0] + rgb[1] * luma_coeff[1] + rgb[2] * luma_coeff[2]
    formula = [luma, luma, luma]
    return np.array(formula)

luma_coeff = [0.2126, 0.7152, 0.0722]

def get_AVGall(arr):
    return np.mean(np.concatenate((arr[:10], arr[-10:])))

class Image:
    
    def __init__(self, fn='', fromArray=np.array([])):
        assert fn or fromArray.ndim>=2, "Image must take a filename or a numpy array as a parameter"
        if fn:
            img = SimpleITK.ReadImage(fn)
            self.__filename = fn
            self.__width = img.GetWidth()
            self.__height = img.GetHeight()
            self.__array = SimpleITK.GetArrayFromImage(img)
            self.__array = self.__array / 65535.0
        else:
            self.__filename = None
            self.__width = fromArray.shape[1]
            self.__height = fromArray.shape[0]
            self.__array = fromArray / 65535.0
    
    def get_filename(self):
        # permet d'obtenir le nom du fichier image origine (s'il existe)
        return self.__filename
    
    def get_array(self):
        # getter pour l'array correspondant à l'image
        return self.__array
    
    def get_grey_array(self, arr):
        # permet d'obtenir la version grise d'un array
        grey_array = []
        for i in range(len(arr)): # not optimized
            inter = []
            for j in range(len(arr[i])):
                inter.append(rgb2grey(arr[i][j]))
            grey_array.append(np.array(inter))
        return np.array(grey_array)
    
    def wiener_filtered(self, arr):
        return wiener(arr)
    
    def __correct_column(self, col):
        # permet de corriger une colonne d'un array avec la méthode de streks correction
        arr = []
        start = np.mean(col[:10])
        end = np.mean(col[-10:])
        for i in range(10, len(col)-10):
            arr.append(col[i] * get_AVGall(self.get_array()) / (((len(col) - i) / len(col)) * start + (i / len(col)) * end))
        return np.concatenate((col[:10], np.array(arr), col[-10:]))

    def streaks_corrected(self, arr):
        # corrige un array en appliquant la streaks correction
        return np.apply_along_axis(self.__correct_column, 0, arr)
    
    def get_width(self):
        # getter pour la largeur de l'image
        return self.__width
    
    def get_height(self):
        # getter pour la hauteur de l'image
        return self.__height
    
    def show(self, mode=''):
        # affiche l'image avec le(s) modes désirés
        # 'r' pour afficher l'image à l'envers
        # 'w' pour que l'image soit passée à travers un filtre de Weiner
        # 'g' pour afficher l'image en gris (avec les coefficients luma)
        # 's' pour réaliser une streaks correction sur l'image
        
        array = self.get_array()
        dir = 'upper'
        
        if 'r' in mode:
            dir ='lower'
        
        if 'w' in mode:
            array = self.wiener_filtered(array)
        if 'g' in mode:
            array = self.get_grey_array(array)
        if 's' in mode:
            array = self.streaks_corrected(array)
        
        array[array > 1] = 1.0
        plt.imshow(array, origin=dir)

In [ ]:
# FUNCTIONS #

def get_med_img(arr):
    """
        Creates a median array from a list of arrays
        
        :param arr: a list of arrays
        :type arr: list or numpy.ndarray
        :return: the median array
        :rtype: numpy.ndarray
        
        :Example:
        
        >>> import numpy
        >>> a = numpy.array([[7,8,9],[16,17,18]])
        >>> b = numpy.array([[13,14,15],[4,5,6]])
        >>> c = numpy.array([[1,2,3],[10,11,12]])
        >>> print(get_med_img([a, b, c]))
        [[ 7.  8.  9.]
         [10. 11. 12.]]
    """
    l3d = np.array(arr)
    return np.median(l3d, axis=0)

In [ ]:
# ACTUAL CLASS #

class Image3d:
    
    def __init__(self, fns, path=''):
        arrays = []
        for i in range(len(fns)):
            img = SimpleITK.ReadImage(path + fns[i])
            arrays.append(SimpleITK.GetArrayFromImage(img))
        self.__arrays = arrays
    
    def get_2dImage(self):
        # retourne l'image 2D correspondant à l'image médiane
        return Image(fromArray=get_med_img(self.get_array()))
    
    def get_width(self):
        # getter pour la largeur de l'image
        return self.get_2dImage().get_width()
    
    def get_height(self):
        # getter pour la hauteur de l'image
        return self.get_2dImage().get_height()
    
    def get_array(self):
        # getter pour l'array correspondant à l'image 3D
        return self.__arrays

In [ ]:
# TESTING #

files = ['scan1.tif', 'scan2.tif', 'scan3.tif', 'scan4.tif', 'scan5.tif']
img3d = Image3d(files, 'tif1/')
img3d.get_2dImage().show('rs')